In [ ]:
from transformers import GPT2Tokenizer, GPT2Model, TrainingArguments, Trainer
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from torch.optim import Adam
from torch.optim import lr_scheduler
from sklearn.metrics import accuracy_score, f1_score
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
pip install transformers[torch]

In [ ]:
pip install accelerate -U

In [ ]:
# Load model and tokenizer

model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# Load data
df = pd.read_csv("lyrics_preprocessed.csv")
lyrics = df['lyrics'].values
mood = df['Mood_encod'].values

tokenizer.pad_token = tokenizer.eos_token

tokenized_data = tokenizer(df['lyrics'].tolist(), truncation=True, padding=True, return_tensors='pt')

dataset = tokenized_data
dataset['labels'] = torch.tensor(df['Mood_encod'].tolist())

# Split dataset
train_dataset = dataset.shuffle(seed=42).select(range(0,1512))
val_dataset = dataset.shuffle(seed=42).select(range(1512,1890))

class GPT2MoodClassifier(nn.Module):
    def __init__(self, gpt2_model, num_classes=4):
        super(GPT2MoodClassifier, self).__init__()
        self.gpt2_model = gpt2_model
        self.classification_head = nn.Linear(gpt2_model.config.hidden_size, num_classes)

    def forward(self, input_ids):
        outputs = self.gpt2_model(input_ids)
        last_hidden_state = outputs[0]
        cls_hidden_state = last_hidden_state[:,0,:]
        logits = self.classification_head(cls_hidden_state)
        return logits

gpt2_model = GPT2Model.from_pretrained(model_name)

num_classes = len(set(mood))

model = GPT2MoodClassifier(gpt2_model, num_classes=num_classes)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

training_args = TrainingArguments("checkpoint_path", num_train_epochs=10, per_device_eval_batch_size=4,\
                                  per_device_train_batch_size=4, evaluation_strategy="epoch",\
                                  gradient_accumulation_steps=4, learning_rate=1e-3, fp16=True)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    pred = np.argmax(logits, axis=-1)
    return {'accuracy_score': accuracy_score(labels, pred)}

class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        return nn.CrossEntropyLoss()(outputs.logits, labels)

trainer = Trainer(model=model, args=training_args, train_dataset=train_dataset, \
                  eval_dataset=val_dataset, compute_metrics=compute_metrics)

trainer.train()